# Plan and Execute Agent

Based on: https://github.com/langchain-ai/langgraph/blob/main/examples/plan-and-execute/plan-and-execute.ipynb

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv('../.env')

True

In [24]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_agent_executor
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Tuple, Annotated, TypedDict
from langchain_core.pydantic_v1 import BaseModel
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate
import operator
from langchain.chains.openai_functions import create_openai_fn_runnable
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

# Step 1: Tools and execution agent

In [8]:
tools = [TavilySearchResults(max_results=3)]

In [9]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4-turbo-preview")
# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [10]:
agent_executor = create_agent_executor(agent_runnable, tools)

In [11]:
agent_executor.invoke(
    {"input": "who is the winnner of the us open", "chat_history": []}
)

{'input': 'who is the winnner of the us open',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': "The winner of the 2023 US Open men's singles is Novak Djokovic. He won his 24th Grand Slam singles title by defeating Daniil Medvedev with a score of 6-3, 7-6(5), 6-3 in the final. This victory ties Margaret Court's record and bolsters Djokovic's case to be considered the greatest tennis player of all time."}, log="The winner of the 2023 US Open men's singles is Novak Djokovic. He won his 24th Grand Slam singles title by defeating Daniil Medvedev with a score of 6-3, 7-6(5), 6-3 in the final. This victory ties Margaret Court's record and bolsters Djokovic's case to be considered the greatest tennis player of all time."),
 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'US Open 2023 winner'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'US Open 2023 winner'}`\n\n\n", message_log=[AIMessage(con

## Step 2: Planning

In [12]:
# holds state: past steps are a tuple of step:result. response is the final response
class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [14]:
class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

The planner is another agent.

In [16]:
planner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

{objective}"""
)
planner = create_structured_output_runnable(
    Plan, ChatOpenAI(model="gpt-4-turbo-preview", temperature=0), planner_prompt
)

In [17]:
planner.invoke(
    {"objective": "what is the hometown of the current Australia open winner?"}
)

Plan(steps=['Identify the current year to determine the most recent Australia Open tournament.', "Search for the winner of the most recent Australia Open in either the men's or women's singles category.", 'Find the birthplace or the commonly known hometown of the identified winner.', 'The hometown of the current Australia Open winner is the answer.'])

## Re-Plan Step

In [19]:
# replanner is yet another agent that re-does the plan based on the result of the previous step
class Response(BaseModel):
    """Response to user."""

    response: str

In [20]:
replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)


replanner = create_openai_fn_runnable(
    [Plan, Response],
    ChatOpenAI(model="gpt-4-turbo-preview", temperature=0),
    replanner_prompt,
)

# The final graph

In [23]:
async def execute_step(state: PlanExecute):
    task = state["plan"][0]
    agent_response = await agent_executor.ainvoke({"input": task, "chat_history": []})
    return {
        "past_steps": (task, agent_response["agent_outcome"].return_values["output"])
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"objective": state["input"]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output, Response):
        return {"response": output.response}
    else:
        return {"plan": output.steps}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return True
    else:
        return False

In [25]:
# the final graph glues together the agents and the state

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.set_entry_point("planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    {
        # If `tools`, then we call the tool node.
        True: END,
        False: "agent",
    },
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [26]:
config = {"recursion_limit": 50}
inputs = {"input": "what is the hometown of the 2024 Australia open winner?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)


{'plan': ['Wait until the 2024 Australian Open concludes.', 'Identify the winner of the 2024 Australian Open.', "Research the winner's biography to find their hometown."]}
{'past_steps': ('Wait until the 2024 Australian Open concludes.', "I'm here to help with information, but I can't wait in real-time or provide updates after the conclusion of the 2024 Australian Open. However, I can help you find information on how to follow the event or provide details and historical context about the tournament. Let me know how I can assist you further!")}
{'plan': ['Identify the winner of the 2024 Australian Open.', "Research the winner's biography to find their hometown."]}
{'past_steps': ('Identify the winner of the 2024 Australian Open.', "The winner of the 2024 Australian Open men's singles title is Jannik Sinner. He won his first ever Grand Slam title by defeating Daniil Medvedev in the final with scores of 3-6, 3-6, 6-4, 6-4, 6-3.")}
{'plan': ["Research Jannik Sinner's biography to find his 